## Standard  stimulus evoked fMRI analysis

In [2]:
import os, shutil, sys, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import helperfunctionsfMRI as fh
import MRIhelperfunctions as h

importlib.reload(fh)
import helperfunctionsfMRI as fh
importlib.reload(h)
import MRIhelperfunctions as h

# Settings

In [5]:
paradigm = {}
folders = {}
averaging = {}

paradigm['TR'] = 1 # in seconds

paradigm['initial_cutoff'] = 30 # number of volumes to remove (in absence of dummy scans).

blurFWHM = 0.3 # In [mm]. Rule-of-thumb: 1.5x the voxel resolution.

highpass = 180 # period [s]. Any signal slower than this will be removed. Should be at least 2x inter-stim. interval


# --- For averaging of blocks:

averaging['baseline'] = 20 # baseline to consider before each block

averaging['time'] = 60 # how many seconds to display after end of stim_block


#======================================================================
folders['animal'] = 'media/sf_MRIDATA/20181214'
folders['scanNumber'] = '8'
#======================================================================


# --- EPI Template for coregistration ---
folders['template'] = '/home/felix/Desktop/examplescan/template/template.nii'


# --- Optional: ROI file:
folders['rois'] = '/home/felix/Desktop/examplescan/template/ROIs.nii'



# --- Define the stimulation paradigm (all units in seconds):
paradigm['baseline'] = 120
paradigm['stim_duration'] = 8 
paradigm['Nblocks'] = 10
paradigm['ISI'] = 90  # Inter-stimulus interval (on+off period)
paradigm['frequency'] = 4 


paradigm['stim_times'], paradigm['stim_onsets'], paradigm['stim_times_all'], paradigm['singleblock'] = fh.CreateStimParadigm(paradigm)

stimulus times: [120, 210, 300, 390, 480, 570, 660, 750, 840, 930]


In [6]:
# this creates also the 'analysis' and 'scan' folders
folders = h.check_and_convert_single(folders)

Bru2 -a -o media/sf_MRIDATA/20181214/analysis/8 media/sf_MRIDATA/20181214/8/subject


# Data Preprocessing

In [4]:
folders['cut'] = fh.initial_cutoff(folders['raw'], paradigm['initial_cutoff'])

KeyError: 'raw'

In [ ]:
# takes several minutes
folders['coreg'] = h.coreg_epi_to_template(folders['template'], folders['cut'])

### Check for motion

In [ ]:
frame = pd.read_csv(os.path.join(folders['scan'],'dfile.1D'), sep=r'\s{2,}', engine='python')

%matplotlib qt
# frame.plot(subplots=True, layout=(9,1))
frame.iloc[:,1:7].plot(subplots=True, layout=(6,1))

High-pass filter, smoothing

In [ ]:
folders['cleaned'] = fh.filter_blur(folders['coreg'], highpass, blurFWHM)

# GLM Analysis

In [ ]:
fh.Deconvolve(folders['cleaned'],paradigm)

# ROI Analysis

Can be run separately on previously analyzed datasets, but first execute all cells under "Settings" above

In [ ]:
folders['cleaned'] = fh.find_cleaned_data(folders)
folders['roidata'] = h.extract_roi(folders['cleaned'],folders['rois'])

roi_df = pd.read_csv(folders['roidata'], sep='\t')
del roi_df['File'], roi_df['Sub-brick']

Plotting whole time course

In [ ]:
%matplotlib qt

plt.figure()
for rois in roi_df.columns:
    plt.plot(roi_df[rois])
_ = [plt.axvline(_p, alpha=0.05, color='red') for _p in paradigm['stim_times_all']]
_ = plt.legend(roi_df.columns)

Plotting averages across stimulus blocks

In [ ]:
%matplotlib inline

roimeans = fh.averageblocks(roi_df, paradigm, averaging)

plt.figure()
_ = [plt.axvline(_p, alpha=0.05, color='red') for _p in paradigm['singleblock']]
plt.axhline(0,color='k')
plt.plot(roimeans)
_ = plt.legend(roi_df.columns)
